In [2]:
from API import PySimFin


In [ ]:
import pandas as pd

In [3]:
simfin = PySimFin("33cd76b1-b978-4165-8b91-5696ddea452a")

In [ ]:
from datetime import datetime, timedelta

In [ ]:
yesterday = (datetime.today()-timedelta(days=1)).strftime("%Y-%m-%d")
yesterday

'2025-03-11'

In [ ]:
#if we want to predict tomorrow's price, we need to pull at least 5 stock days, so that last row has all previous day values
AAPL = simfin.get_share_prices("AAPL", "2025-03-01", "2025-03-11")

2025-03-12 09:50:13,435 - INFO - Fetching share prices for AAPL from 2025-03-01 to 2025-03-11.


In [ ]:
AAPL

,name,id,ticker,currency,isin,Date,Dividend Paid,Common Shares Outstanding,Last Closing Price,Adjusted Closing Price,Highest Price,Lowest Price,Opening Price,Trading Volume
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-03,None,15022073000,238.03,238.03,244.03,236.11,241.79,46873613
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-04,None,15022073000,235.93,235.93,240.07,234.68,237.71,53009183
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-05,None,15022073000,235.74,235.74,236.55,229.23,235.42,46987098
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-06,None,15022073000,235.33,235.33,237.86,233.16,234.44,44856248
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-07,None,15022073000,239.07,239.07,241.37,234.76,235.10,45602641
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-10,None,15022073000,227.48,227.48,236.16,224.22,235.54,71451281
0,APPLE INC,111052,AAPL,USD,US0378331005,2025-03-11,None,15022073000,220.84,220.84,225.84,217.45,223.81,73971209


In [ ]:
# drop irrelevant columns

AAPL = AAPL[["ticker","Date","Adjusted Closing Price"]]
AAPL

,ticker,Date,Adjusted Closing Price
0,AAPL,2025-03-03,238.03
0,AAPL,2025-03-04,235.93
0,AAPL,2025-03-05,235.74
0,AAPL,2025-03-06,235.33
0,AAPL,2025-03-07,239.07
0,AAPL,2025-03-10,227.48
0,AAPL,2025-03-11,220.84


In [ ]:
# do some data cleaning 

AAPL["Date"] = pd.to_datetime(AAPL["Date"], format="%Y-%m-%d")
AAPL.rename(columns={"Adjusted Closing Price": "Price"}, inplace=True)
AAPL.rename(columns={"ticker": "Ticker"}, inplace=True)


In [ ]:
AAPL.head()

,Ticker,Date,Price
0,AAPL,2025-03-03,238.03
0,AAPL,2025-03-04,235.93
0,AAPL,2025-03-05,235.74
0,AAPL,2025-03-06,235.33
0,AAPL,2025-03-07,239.07


In [ ]:
# Add last 4 days' prices
for day in range(1, 5):
    AAPL[f"Price d-{day}"] = AAPL.groupby("Ticker")["Price"].shift(day)
        

In [ ]:
AAPL

,Ticker,Date,Price,Price d-1,Price d-2,Price d-3,Price d-4
0,AAPL,2025-03-03,238.03,NaN,NaN,NaN,NaN
0,AAPL,2025-03-04,235.93,238.03,NaN,NaN,NaN
0,AAPL,2025-03-05,235.74,235.93,238.03,NaN,NaN
0,AAPL,2025-03-06,235.33,235.74,235.93,238.03,NaN
0,AAPL,2025-03-07,239.07,235.33,235.74,235.93,238.03
0,AAPL,2025-03-10,227.48,239.07,235.33,235.74,235.93
0,AAPL,2025-03-11,220.84,227.48,239.07,235.33,235.74


In [ ]:
# Drop missing values
AAPL = AAPL.dropna()
        

In [ ]:
# Drop "Date" and "Ticker"

AAPL = AAPL.drop(columns=["Ticker","Date"])

In [ ]:
AAPL

,Price,Price d-1,Price d-2,Price d-3,Price d-4
0,239.07,235.33,235.74,235.93,238.03
0,227.48,239.07,235.33,235.74,235.93
0,220.84,227.48,239.07,235.33,235.74


Load model:

In [ ]:
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [ ]:
#hardcoding this for AAPL
model_path = "Scripts/models/model_AAPL.pkl"
scaler_path = "Scripts/models/scaler_AAPL.pkl"

In [ ]:
# Load trained model and scaler for this ticker
if not os.path.exists(model_path) or not os.path.exists(scaler_path):
    raise FileNotFoundError("Model or Scaler not found. Train the model first.")

model = joblib.load(model_path)
scaler = joblib.load(scaler_path)
print(f"Model and scaler for ticker loaded successfully!\n") #add ticker as variable here for script

Model and scaler for ticker loaded successfully!



In [ ]:
# Make a prediction for the next day's price movement
if model is None or scaler is None:
    raise ValueError("Model and scaler not loaded. Run load_model() first.")

# Use the last available row for prediction
latest_features = AAPL.iloc[-1:]
latest_features_scaled = scaler.transform(latest_features)

prediction = model.predict(latest_features_scaled)
print("UP" if {prediction[0] == 1} else "DOWN")

UP
